In [1]:
%matplotlib inline
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import math
import time
import numpy as np
import tensorflow as tf

from d2l import tensorflow as d2l
import timeit
from datetime import datetime


# 1.图计算
0. eager execution即刻计算：  
由python解释器执行python代码
1. graph execution图计算：  
一、构建计算图（可以由python代码构建） -->  tf.Graph  
tf.Graph组成：  
    tf.Operation（tf操作，unit节点）  
    tf.Tensor（tf数据，在节点中流动）  
二、编译计算图  
编译好的计算图可以用于没有python解释器的设备（比如手机等）  
同时计算图还可以在多个device之间分隔  
加速计算  

## 如何用Python构建计算图

### 使用tf.function修饰器，将python function转化为graphs
tf.function修饰器在构建计算图时  
python function内部的inner function也会被包含在计算图内部  
python function内部的tf.Operations直接被转化为tf.graph  
python function内部的python logic，比如if-else, loops, break, return, continue等，被转化为tf.autograph

In [ ]:
# Define a Python function.
def a_regular_function(x, y, b):
    x = tf.matmul(x, y)
    x = x + b # inner function也被包含在tf.graph内
    return x

# `a_function_that_uses_a_graph` is a TensorFlow `Function`.
# tf函数（计算图）
a_function_that_uses_a_graph = tf.function(a_regular_function)

x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = a_regular_function(x1, y1, b1).numpy()
# tf函数（计算图）可以像函数一样调用
tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()
assert(orig_value == tf_function_value)

In [3]:
def simple_relu(x):
  if tf.greater(x, 0.): # python logic被tf.function处理成tf.autograph
    return x
  else:
    return 2.

# `tf_simple_relu` is a TensorFlow `Function` that wraps `simple_relu`.
# 计算图tf_simple_relu
tf_simple_relu = tf.function(simple_relu)

# print("First branch, with graph:", tf_simple_relu(tf.constant(1)).numpy())
# print("Second branch, with graph:", tf_simple_relu(tf.constant(-1)).numpy())

In [ ]:
# 把python logic处理成autograph的自动代码
# This is the graph-generating output of AutoGraph.
print(tf.autograph.to_code(simple_relu))

### 打印定义好的计算图graph
计算图的.get_concrete_function(input_data).graph.as_graph_def()函数，可以打印具体的图（各个node节点）

In [5]:
# tf_simple_relu目前是经由tf.function生成的计算图
# 查看计算图
print(tf_simple_relu.get_concrete_function(tf.constant(-1.)).graph.as_graph_def())

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "Greater/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 0.0
      }
    }
  }
}
node {
  name: "Greater"
  op: "Greater"
  input: "x"
  input: "Greater/y"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "cond"
  op: "StatelessIf"
  input: "Greater"
  input: "x"
  attr {
    key: "Tcond"
    value {
      type: DT_BOOL
    }
  }
  attr {
    key: "Tin"
    value {
      list {
        type: DT_FLOAT
      }
    }
  }
  attr {
    key: "Tout"
    value {
      list {
        type: DT_FLOAT
        type: DT_BOOL
      }
    

### 计算图的多态性
计算图usr_def_graph执行图计算：  
执行图计算的方式就像函数调用，即 usr_def_graph(input_data)  

定义好的tf.Graph（内存中的），dataflow只能运行其signature定义好的数据类型。但是计算图（概念中的）可能可以支持多种数据类型。  
所以：  
当输入旧数据类型的data时，传入signature为该数据类型的tf.Graph  
当输入新数据类型的data时，根据计算图（概念中的）创建新的tf.Graph

In [14]:
# 计算图
@tf.function
def my_relu(x):
  return tf.maximum(0., x)

# `my_relu` creates new graphs as it observes more signatures.
print(my_relu(tf.constant(5.5)))
print(my_relu([2, -2]))
print(my_relu(tf.constant([3., -3.])))

tf.Tensor(5.5, shape=(), dtype=float32)
tf.Tensor([2. 0.], shape=(2,), dtype=float32)
tf.Tensor([3. 0.], shape=(2,), dtype=float32)


In [16]:
# These two calls do *not* create new graphs.
print(my_relu(tf.constant(-2.5))) # Signature matches `tf.constant(5.5)`.
print(my_relu(tf.constant([-1., 1.]))) # Signature matches `tf.constant([3., -3.])`.

tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor([0. 1.], shape=(2,), dtype=float32)


### 打印计算图曾经执行过的数据类型signature
计算图.pretty_printed_concrete_signatures()方法

In [19]:
print(my_relu.pretty_printed_concrete_signatures())

my_relu(x)
  Args:
    x: float32 Tensor, shape=()
  Returns:
    float32 Tensor, shape=()

my_relu(x)
  Args:
    x: float32 Tensor, shape=(2,)
  Returns:
    float32 Tensor, shape=(2,)

my_relu(x=[2, -2])
  Returns:
    float32 Tensor, shape=(2,)


## 图计算和即刻计算
如果使用tf.function构建计算图usr_def_func之后，usr_def_func也可以切换成即刻计算，只需：  
tf.config.run_functions_eagerly(True)

In [22]:
@tf.function
def get_MSE(y_true, y_pred):
    sq_diff = tf.pow(y_true - y_pred, 2)
    return tf.reduce_mean(sq_diff)

y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)

In [23]:
# graph execute
# 图计算，直接调用即可
get_MSE(y_true, y_pred)

<tf.Tensor: shape=(), dtype=int32, numpy=22>

In [24]:
# eager execute
# 即刻计算
tf.config.run_functions_eagerly(True)
get_MSE(y_true, y_pred)

<tf.Tensor: shape=(), dtype=int32, numpy=22>

In [25]:
tf.config.run_functions_eagerly(False)

### Trace
在构建计算图（一般来说是第一次调用usr_def_graph时）的时候，tf会抓取函数中的tf op和python logic来构建计算图 ---> 这个过程叫做trace  
于是其他的python函数比如print()，不会被tf抓取到计算图中  
计算图构建完毕之后，再调用usr_def_graph时，tf将直接运行计算图，而不是执行python代码。于是代码中的print()函数将不会被执行

### Non-strict Execution
图计算执行过程中，不是所有的tf op都会被执行，只会执行必要的node节点。  
程序的效果有：
1、The return value of the function  
2、Documented well-known side-effects such as:  
        Input/output operations, like tf.print  
        Debugging operations, such as the assert functions in tf.debugging  
        Mutations of tf.Variable  

### 图计算的性能
构建图比较费时间，但是图计算比python解释运算要快速

In [26]:
x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)

def power(x, y):
  result = tf.eye(10, dtype=tf.dtypes.int32)
  for _ in range(y):
    result = tf.matmul(x, result)
  return result

In [27]:
print("Eager execution:", timeit.timeit(lambda: power(x, 100), number=1000))

Eager execution: 1.3119133689997398


In [28]:
power_as_graph = tf.function(power)
print("Graph execution:", timeit.timeit(lambda: power_as_graph(x, 100), number=1000))

Graph execution: 0.3988587320000079


In [29]:
print(power_as_graph.get_concrete_function(x, 3).graph.as_graph_def())

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 10
        }
        dim {
          size: 10
        }
      }
    }
  }
}
node {
  name: "eye/ones"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 10
          }
        }
        int_val: 1
      }
    }
  }
}
node {
  name: "eye/diag/k"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 0
      }
    }
  }
}
node {
  name: "eye/diag/num_rows"
  op: "Const"
  attr {
    key: "dtype"
    value {
    

如何避免多次retrace？  
在usr_def_func中加入print statement。这样每当在构建计算图的时候，print都会被执行，并打印warning。  
避免使用python argument作为计算图的输入。python args很可能会导致构建计算图。